## The Facenet Library
* The case of face detection using Facenet library's models is an interesting case. We don't have to train anything or worry about models or hyperparameters. We simply generate embeddings.
* Embeddings are generating using Facenet's CNN models that is for detedting fac

In [1]:
import os
from PIL import Image
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine

# Initialize MTCNN and InceptionResnetV1
mtcnn = MTCNN(image_size=160, margin=20)
model = InceptionResnetV1(pretrained='vggface2').eval()


In [ ]:

# Path to the dataset containing folders of individuals (for known faces)
dataset_path = "./faces/"  # Replace with your dataset path

# Threshold for similarity matching
threshold = 0.6

# Step 1: Generate embeddings for known faces

known_embeddings = {}

for person_name in os.listdir(dataset_path):
   
    person_folder = os.path.join(dataset_path, person_name)
    
    if os.path.isdir(person_folder):
        for image_name in os.listdir(person_folder):
            
            image_path = os.path.join(person_folder, image_name)
            try:
                # Load image and process with MTCNN
                
                image = Image.open(image_path)
                cropped_face = mtcnn(image)

                if cropped_face is not None:
                    # Generate 128D embeddings for the face
                   
                    embedding = model(cropped_face.unsqueeze(0))
                    known_embeddings[person_name] = embedding.detach()
                    break  # Only use one image per person
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

    




In [3]:

# Example usage
import numpy as np
def recognize_face(test_image_path):
    try:
        # Load test image and process with MTCNN
        test_image = Image.open(test_image_path)
       
        cropped_face = mtcnn(test_image)
        

        if cropped_face is None:
            
            print("No face detected in the image.")
            return

        # Generate embedding for the test image
        test_embedding = np.array(model(cropped_face.unsqueeze(0)).detach()).flatten()
       

        # Compare with known embeddings
        for person_name, known_embedding in known_embeddings.items():
            
            cosine_measure=cosine(test_embedding, np.array(known_embedding).flatten())
           
            similarity=1-cosine_measure
            
            if similarity > threshold:
                print(f"Match found: {person_name} (Similarity: {similarity:.2f})")
                return

        print("No match found.")
    except Exception as e:
        print(f"Error processing test image: {e}")

# Example usage

test_image_path = r"C:\Users\PMLS\OneDrive - Higher Education Commission\Documents\University Work\Sixth Semester\Machine Learning\Assignments\Facenet assignment\faces\Abdullah_Gul\Abdullah_Gul_0014.jpg"  # Replace with the test image path
recognize_face(test_image_path)

Match found: Abdullah_Gul (Similarity: 0.90)


# Conclusion
* The results are comprehensibly correct. 
* Although we have generating embeddings for each person based on a single image which could result in bad representation issue and declined accuracy but the model was checked with 2 random images of 2 people and the results were accurate and the model was able to correctly identify the face.